In [55]:
# ================================
# IMPORTACIÓN DE LIBRERÍAS
# ================================

# spaCy se usa para Procesamiento de Lenguaje Natural (NLP)
import spacy

# requests permite consumir APIs externas (Air-Port-Codes)
import requests

# re se usa para expresiones regulares (fechas, patrones de texto)
import re

# datetime permite manejar y normalizar fechas
from datetime import datetime

# os permite leer variables de entorno (API Key segura)
import os

# ================================
# CARGA DEL MODELO DE NLP EN ESPAÑOL
# ================================

# Modelo pequeño de spaCy para español
# Se usa para tokenización y reconocimiento de entidades
nlp = spacy.load("es_core_news_sm")

# ================================
# VARIABLES GLOBALES
# ================================

# Se obtiene la API KEY desde las variables de entorno
API_KEY = os.getenv("APC_API_KEY")

In [60]:
# Diccionario de meses para convertir texto a número
MESES = {
    "enero":1, "febrero":2, "marzo":3, "abril":4,
    "mayo":5, "junio":6, "julio":7, "agosto":8,
    "septiembre":9, "setiembre":9, "octubre":10,
    "noviembre":11, "diciembre":12
}

# Aerolíneas comunes para mejorar la detección
AEROLINEAS = [
    "Iberia", "Lufthansa", "Air Europa", "AirEuropa",
    "Ryanair", "Vueling", "KLM", "Avianca", "LATAM"
]

# ================================
# FUNCIÓN: TEXTO A NÚMERO
# Convierte números escritos a enteros
# ================================

def texto_a_numero(texto: str) -> int:
    """
    Convierte números escritos en palabras a valores enteros.
    Ejemplo: 'dos' -> 2
    """

    nums = {"un":1,"uno":1,"una":1,"dos":2,"tres":3,"cuatro":4,"cinco":5,"seis":6,"siete":7,"ocho":8,"nueve":9,"diez":10}

    # Si ya es un dígito, se devuelve directamente
    if texto.isdigit():
        return int(texto)

    return nums.get(texto.lower(), 1)

# ================================
# FUNCIÓN: NORMALIZAR FECHA
# Convierte fechas a formato dd-mm-yyyy
# ================================

def normalizar_fecha(frase: str) -> str | None:
    """
    Detecta fechas escritas en lenguaje natural
    y las convierte al formato dd-mm-yyyy.
    """
    fecha = frase.lower()

    # Caso 1: "el 5 de agosto"
    m = re.search(r'(?:el\s+)?(\d{1,2})\s+de\s+([a-záéíóú]+)', fecha)
    if m:
        d = int(m.group(1))
        mes_txt = m.group(2)
        if mes_txt in MESES:
            mes = MESES[mes_txt]
            y = datetime.now().year
            return f"{d:02d}-{mes:02d}-{y}"

    # Caso 2: "en septiembre" → se asume día 01
    m2 = re.search(r'\b(?:en|para)\s+([a-záéíóú]+)\b', fecha)
    if m2:
        mes_txt = m2.group(1)
        if mes_txt in MESES:
            mes = MESES[mes_txt]
            y = datetime.now().year
            return f"01-{mes:02d}-{y}"

    return None

def extraer_cantidad(doc) -> int:
    for token in doc:
        if token.like_num:
            return texto_a_numero(token.text)
    return 1

# ================================
# FUNCIÓN: EXTRAER AEROLÍNEA
# ================================

def extraer_aerolinea(frase: str, doc) -> str | None:
    """
    Identifica la aerolínea mencionada usando:
    1. Lista de aerolíneas conocidas
    2. Entidades ORG detectadas por spaCy
    """

    # A) Búsqueda directa por lista (captura "AirEuropa" aunque spaCy no la marque ORG)
    low = frase.lower()
    for a in AEROLINEAS_CONOCIDAS:
        if a.lower().replace(" ", "") in low.replace(" ", ""):
            # normaliza a la forma bonita
            return "AirEuropa" if a.lower().replace(" ", "") == "aireuropa" else a

    # B) Búsqueda por entidades NLP spaCy ORG
    for ent in doc.ents:
        if ent.label_ == "ORG":
            return ent.text

    return None

def _limpiar_ciudad(txt: str) -> str:
    txt = txt.strip().strip(",. ")
    # Si viene "la" / "el"
    txt = re.sub(r'^(la|el|los|las)\s+', '', txt, flags=re.I)
    return txt

# ================================
# FUNCIÓN: EXTRAER ORIGEN Y DESTINO
# ================================

def extraer_origen_destino(frase: str) -> tuple[str|None, str|None]:
    """
    Casos cubiertos:
      - 'de Madrid a Frankfurt'
      - 'Madrid a Roma'
      - 'a Madrid' (solo destino)
      - 'desde Quito hacia Madrid'
    Soporta ciudades con varias palabras (ej: 'Nueva York')
    """
    s = frase.strip()

    # Captura ciudades con 1+ palabras que empiecen en mayúscula (incluye tildes)
    CIUDAD = r'([A-ZÁÉÍÓÚÑ][\wÁÉÍÓÚÑáéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑ][\wÁÉÍÓÚÑáéíóúñ]+)*)'

    # 1) de X a Y
    m = re.search(rf'\b(?:de|desde)\s+{CIUDAD}\s+\b(?:a|hacia|para)\s+{CIUDAD}\b', s)
    if m:
        return _limpiar_ciudad(m.group(1)), _limpiar_ciudad(m.group(2))

    # 2) X a Y (sin "de")
    m2 = re.search(rf'\b{CIUDAD}\s+\b(?:a|hacia|para)\s+{CIUDAD}\b', s)
    if m2:
        return _limpiar_ciudad(m2.group(1)), _limpiar_ciudad(m2.group(2))

    # 3) solo destino: "a Madrid"
    m3 = re.search(rf'\b(?:a|hacia|para)\s+{CIUDAD}\b', s)
    if m3:
        return None, _limpiar_ciudad(m3.group(1))

    return None, None

# ================================
# FUNCIÓN: CONSUMO DE API AIR-PORT-CODES
# ================================

FALLBACK_IATA = {
    "Quito":"UIO","Guayaquil":"GYE","Madrid":"MAD","Frankfurt":"FRA","Barcelona":"BCN",
    "Roma":"FCO","Berlin":"BER","Sevilla":"SVQ","Miami":"MIA","Bogota":"BOG","Lima":"LIM","Paris":"CDG"
}

_iata_cache = {}  # caché simple en memoria

def obtener_iata(ciudad: str, api_key: str | None = None) -> str:
    """
    term: ciudad o aeropuerto (ej: 'Madrid', 'Frankfurt', 'Sevilla')
    Retorna un IATA (ej: MAD) o 'N/A'
    """
    if not ciudad:
        return "N/A"

    ciudad_t = ciudad.title()
    if not api_key:
        return FALLBACK_IATA.get(ciudad_t, "N/A")

    url = "https://www.air-port-codes.com/api/v1/multi"
    headers = {"APC-Auth": api_key}

    try:
        r = requests.post(url, headers=headers, data={"term": ciudad_t}, timeout=10)
        if r.status_code == 200:
            data = r.json()
            # La respuesta trae 'airports' con objetos que incluyen 'iata' según docs
            if data.get("status") and data.get("airports"):
                return data["airports"][0].get("iata", "N/A") or "N/A"
    except Exception:
        pass

    return FALLBACK_IATA.get(ciudad_t, "N/A")

# ================================
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO
# ================================

def procesar_solicitud(frase: str) -> dict:
    """
    Procesa la frase del usuario y devuelve
    un JSON con la información extraída.
    """
    doc = nlp(frase)

    origen, destino = extraer_origen_destino(frase)
    fecha = normalizar_fecha(frase)
    cantidad = extraer_cantidad(doc)
    aerolinea = extraer_aerolinea(frase, doc)

    return {
        "origen": origen,
        "destino": destino,
        "fecha": fecha,          # ya viene dd-mm-yyyy o None
        "cantidad": cantidad,
        "aerolínea": aerolinea
    }

def construir_json_final(extraido: dict, api_key: str | None = None) -> dict:
    iata_from = obtener_iata(extraido["origen"], api_key) if extraido["origen"] else "N/A"
    iata_to   = obtener_iata(extraido["destino"], api_key) if extraido["destino"] else "N/A"

    return {
        "Origen": extraido["origen"],
        "Ciudad Destino": extraido["destino"],
        "Nombre Ciudad IATA From": iata_from,
        "IATA To": iata_to,
        "Fecha": extraido["fecha"],
        "Pax": int(extraido["cantidad"])
    }

In [61]:
# ================================
# ASISTENTE INTERACTIVO
# ================================

def asistente():
    print('Hola, bienvenido a "TravelBot". ¿Como te puedo ayudar?')
    print("(Escribe 'salir' para terminar)")

    while True:
        entrada = input("\nUsuario: ").strip()
        if entrada.lower() == "salir":
            print("¡Hasta luego!")
            break

        extraido = procesar_solicitud(entrada)
        print("\n--- Out (JSON Extraído) ---")
        print(extraido)

        # Si falta origen o destino, pedirlo
        if not extraido["destino"]:
            print("Bot: Me falta el destino. Ejemplo: 'a Madrid' o 'de Quito a Madrid'.")
            continue
        if not extraido["origen"]:
            print("Bot: Me falta el origen. Ejemplo: 'de Quito a Madrid'.")
            continue

        print(f'\nBot: Perfecto, Comienzo la búsqueda de tu viaje a {extraido["destino"]} '
              f'desde {extraido["origen"]} para el {extraido["fecha"]} con {extraido["aerolínea"]}.')

        final = construir_json_final(extraido, API_KEY)
        print("\n--- JSON FINAL PARA API DE RESERVAS ---")
        print(final)

asistente()

Hola, bienvenido a "TravelBot". ¿Como te puedo ayudar?
(Escribe 'salir' para terminar)

Usuario: Comprar tres billetes para el 15 de octubre con Iberia de Quito a Madrid

--- Out (JSON Extraído) ---
{'origen': 'Quito', 'destino': 'Madrid', 'fecha': '15-10-2026', 'cantidad': 3, 'aerolínea': 'Iberia'}

Bot: Perfecto, Comienzo la búsqueda de tu viaje a Madrid desde Quito para el 15-10-2026 con Iberia.

--- JSON FINAL PARA API DE RESERVAS ---
{'Origen': 'Quito', 'Ciudad Destino': 'Madrid', 'Nombre Ciudad IATA From': 'UIO', 'IATA To': 'MAD', 'Fecha': '15-10-2026', 'Pax': 3}

Usuario: salir
¡Hasta luego!


In [62]:
entradas = [
    "Quiero 2 billetes de Madrid a Frankfurt en Septiembre",
    "Necesito comprar un billete a Madrid el 5 de Agosto",
    "Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia",
    "Billete barato AirEuropa de Madrid a Sevilla"
]

for e in entradas:
    ex = procesar_solicitud(e)
    print("\nIN:", e)
    print("OUT:", ex)
    print("FINAL:", construir_json_final(ex))




IN: Quiero 2 billetes de Madrid a Frankfurt en Septiembre
OUT: {'origen': 'Madrid', 'destino': 'Frankfurt', 'fecha': '01-09-2026', 'cantidad': 2, 'aerolínea': None}
FINAL: {'Origen': 'Madrid', 'Ciudad Destino': 'Frankfurt', 'Nombre Ciudad IATA From': 'MAD', 'IATA To': 'FRA', 'Fecha': '01-09-2026', 'Pax': 2}

IN: Necesito comprar un billete a Madrid el 5 de Agosto
OUT: {'origen': None, 'destino': 'Madrid', 'fecha': '05-08-2026', 'cantidad': 5, 'aerolínea': None}
FINAL: {'Origen': None, 'Ciudad Destino': 'Madrid', 'Nombre Ciudad IATA From': 'N/A', 'IATA To': 'MAD', 'Fecha': '05-08-2026', 'Pax': 5}

IN: Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia
OUT: {'origen': 'Barcelona', 'destino': 'Roma', 'fecha': '25-08-2026', 'cantidad': 25, 'aerolínea': 'Iberia'}
FINAL: {'Origen': 'Barcelona', 'Ciudad Destino': 'Roma', 'Nombre Ciudad IATA From': 'BCN', 'IATA To': 'FCO', 'Fecha': '25-08-2026', 'Pax': 25}

IN: Billete barato AirEuropa de Madrid a Sevilla
OUT: {'origen': 'Madrid

In [64]:
print(obtener_iata("Madrid"))
print(obtener_iata("Frankfurt"))
print(obtener_iata("Paris"))


MAD
FRA
CDG
